In [1]:
import boto3
import json
import os
import mysql.connector

## Initialize MySQL database 

In [9]:
import boto3

# Create RDS instance
rds = boto3.client('rds')

try:
    response = rds.create_db_instance(
        DBInstanceIdentifier='autobiography-db',
        DBName='autobiography',
        MasterUsername='username',
        MasterUserPassword='password',
        DBInstanceClass='db.t3.micro',
        Engine='MySQL',
        AllocatedStorage=5
    )

    # Wait until DB is available to continue
    rds.get_waiter('db_instance_available').wait(DBInstanceIdentifier='autobiography-db')

except rds.exceptions.DBInstanceAlreadyExistsFault:
    print("Database instance already exists.")

# Describe where DB is available and on what port
try:
    db = rds.describe_db_instances(DBInstanceIdentifier='autobiography-db')['DBInstances'][0]
    ENDPOINT = db['Endpoint']['Address']
    PORT = db['Endpoint']['Port']
    DBID = db['DBInstanceIdentifier']

    print(DBID, "is available at", ENDPOINT, "on Port", PORT)
except Exception as e:
    print("Error retrieving database information:", e)

Database instance already exists.
autobiography-db is available at autobiography-db.cfgznmmh4qfo.us-east-1.rds.amazonaws.com on Port 3306


In [10]:
# Get Name of Security Group
SGNAME = db['VpcSecurityGroups'][0]['VpcSecurityGroupId']

# Adjust Permissions for that security group so that we can access it on Port 3306
# If already SG is already adjusted, print this out
try:
    ec2 = boto3.client('ec2')
    data = ec2.authorize_security_group_ingress(
            GroupId=SGNAME,
            IpPermissions=[
                {'IpProtocol': 'tcp',
                 'FromPort': PORT,
                 'ToPort': PORT,
                 'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
            ]
    )
except ec2.exceptions.ClientError as e:
    if e.response["Error"]["Code"] == 'InvalidPermission.Duplicate':
        print("Permissions already adjusted.")
    else:
        print(e)

Permissions already adjusted.


In [15]:
import mysql.connector

# Database connection configuration
config = {
    'user': 'username',
    'password': 'password',
    'host': 'autobiography-db.cfgznmmh4qfo.us-east-1.rds.amazonaws.com',
    'database': 'autobiography',
    'raise_on_warnings': True
}

try:
    # Connect to the database
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()

    # SQL to create the 'books' and 'persons' tables without foreign keys
    create_books_table = """
    CREATE TABLE books (
        book_title VARCHAR(255) PRIMARY KEY,
        Extroversion FLOAT,
        Neuroticism FLOAT,
        Agreeableness FLOAT,
        Conscientiousness FLOAT,
        Openness FLOAT,
        person_name VARCHAR(255) NULL  
    );
    """
    create_persons_table = """
    CREATE TABLE persons (
        person_name VARCHAR(255) PRIMARY KEY,
        Extroversion FLOAT,
        Neuroticism FLOAT,
        Agreeableness FLOAT,
        Conscientiousness FLOAT,
        Openness FLOAT,
        book_title VARCHAR(255) NULL 
    );
    """

    # Execute SQL commands to create tables
    cursor.execute(create_books_table)
    cursor.execute(create_persons_table)

    # Commit changes
    conn.commit()
    print("Tables created successfully.")

except mysql.connector.Error as err:
    print("Error occurred:", err)

finally:
    if conn.is_connected():
        cursor.close()
        conn.close()
        print("Connection closed.")

Tables created successfully.
Connection closed.


# Initialize S3 to store the raw data

In [14]:
# create s3 bucket
bucket_name = 'autobiography-raw-data'
region = 'us-east-1'
s3 = boto3.client('s3', region_name=region if region else 'us-east-1')
try:
    if region is None or region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
        )
    print(f"Bucket {bucket_name} created successfully.")
except s3.exceptions.BucketAlreadyExists as e:
    print(f"Bucket {bucket_name} already exists.")
except s3.exceptions.BucketAlreadyOwnedByYou as e:
    print(f"Bucket {bucket_name} already owned by you.")
except Exception as e:
    print(f"An error occurred: {e}")

Bucket autobiography-raw-data created successfully.
